Imports

In [174]:
import numpy as np
import klayout.db as db

Parameters for grating

In [175]:
## Yue Wang
folder_name = 'YueWang'
feature_size = 0.150

save_name = 'YW_Px410-Py320-50um'
widths = [0, 50]
heights = [0, 50]
y_periods = [0.320, 0.320]
x_periods = [0.410, 0.410]

# save_name = 'YW_Px420-Py325-100um'
# widths = [0, 100]
# heights = [0, 100]
# y_periods = [0.325, 0.325]
# x_periods = [0.420, 0.420]

# save_name = 'YW_Px420-Py325-200um'
# widths = [0, 200]
# heights = [0, 200]
# y_periods = [0.325, 0.325]
# x_periods = [0.420, 0.420]

# save_name = 'YW_Px420-Py325-400um'
# widths = [0, 400]
# heights = [0, 400]
# y_periods = [0.325, 0.325]
# x_periods = [0.420, 0.420]

cell_name = save_name

layer_number = 0
layer_dose = 1.0 # Check this works in Voyager, multiplier may be incorrect

# ONLY WORKS WITH TWO PERIODS IN EITHER X OR Y OR BOTH, MORE THAN TWO WILL NOT TILE CORRECTLY
# widths = [0, 500]
# heights = [0, 500]
# y_periods = [0.320, 0.320]
# x_periods = [0.400, 0.400]
radius = feature_size/2
num_of_vertices = 16
# ffs = [0.5, 0.5] # Not implemented

squares = True

grating_x = 0
grating_y = 0

# 'num_of_periods' needs to be even so that the grating can be generated 
# around the centre point, this is required due to the change in period
# if we want the pattern to be 'bow-tie' shaped
num_of_x_periods = round((max(widths) - min(widths)) / max(x_periods))
if num_of_x_periods % 2 == 1:
    num_of_x_periods = num_of_x_periods - 1

num_of_y_periods = round((max(heights) - min(heights)) / max(y_periods))
if num_of_y_periods % 2 == 1:
    num_of_y_periods = num_of_y_periods - 1

print(f'Num of periods : ({num_of_x_periods}, {num_of_y_periods})')

Num of periods : (122, 156)


KLayout set up

*Note: A new layout must be defined if changes in grating parameters are made*

In [176]:
layout = db.Layout()

# sets the database unit to 1 um, base usits seem to be millimeters.
layout.dbu = 0.001

cell = layout.create_cell(cell_name)
layer = layout.layer(layer_number, int(layer_dose * 1000)) # Check dose multiplier

Determine centre-point of each hole.  Translate all coordinates to final location.

In [177]:
xs = []
ys = []
rs = []

for idx in range(1, len(x_periods)):
    # first row x-coordinates
    first_x = np.arange(0, (num_of_x_periods * x_periods[idx-1]) + x_periods[idx-1], x_periods[idx-1])
    first_x = first_x[:num_of_x_periods] # force array to be 'num_of_periods' long

    # last row x-coordinates
    last_x = np.arange(0, (num_of_x_periods * x_periods[idx])+ x_periods[idx], x_periods[idx])
    last_x = last_x[:num_of_x_periods] # force array to be 'num_of_periods' long

    for fx, lx in zip(first_x, last_x):
        xs.extend(list(np.linspace(fx, lx, num_of_y_periods) + grating_x))
        rs.extend(list(np.ones(num_of_x_periods) * radius))

for idx in range(1, len(y_periods)):
    # first column y-coordinates
    first_y = np.arange(0, (num_of_y_periods * y_periods[idx-1]) + y_periods[idx-1], y_periods[idx-1])
    first_y = first_y[:num_of_y_periods] # force array to be 'num_of_periods' long

    # last column y-coordinates
    last_y = np.arange(0, (num_of_y_periods * y_periods[idx]) + y_periods[idx], y_periods[idx])
    last_y = last_y[:num_of_y_periods] # force array to be 'num_of_periods' long

    for fy, ly in zip(first_y, last_y):
        ys.extend(list(np.linspace(fy, ly, num_of_x_periods) + grating_y))

# As built the 'ys' array is not 'interleved' correctly with the xs array, this is because the 'linspace'
# array is built in one go for both x and y.  To fix this, the 'ys' array needs to be rotated
# however the arrays are actually 1D vectors, so first I reshape the array into a square with side length
# 'num_of_x_periods' then 'transpose' to rotate the array, then 'ravel' to make it into a
# 1D vector for further processing
ys = list(np.asarray(ys).reshape(-1, num_of_x_periods).transpose().ravel())

print(list(zip(xs, ys, rs))[0:5])

[(0.0, 0.0, 0.075), (0.0, 0.32, 0.075), (0.0, 0.64, 0.075), (0.0, 0.96, 0.075), (0.0, 1.28, 0.075)]


Concactenate x and y coordinates, use them to create a 'DPoint' then use these to create 'DPolygon's.  Adding these polygons to the cell called 'top'

In [178]:
for x, y, r in zip(xs, ys, rs):
    if squares:
        c = db.DBox(x-r, y-r, x+r, y+r)
    else:
        c = db.DPolygon.ellipse(db.DBox(x-r, y-r, x+r, y+r), num_of_vertices)
    cell.shapes(layer).insert(c)

Save out the completed GDSII file

In [179]:
layout.write(f'{folder_name}/{save_name}.gds')